# Modelos NLP con TF-IDF para Detección de Desinformación

En este notebook implemento el análisis de procesamiento de lenguaje natural usando TF-IDF y modelos tradicionales de machine learning. Mi objetivo es crear un sistema robusto para detectar noticias falsas basándome únicamente en el contenido textual.

## Mi Approach de NLP

Voy a seguir estos pasos principales:

1. **Preprocesamiento de texto**: Limpio y normalizo el texto eliminando ruido
2. **Corrección de data leakage**: Elimino duplicados para evitar sobreajuste
3. **Vectorización TF-IDF**: Convierto texto a representación numérica
4. **Modelos tradicionales**: Pruebo LR, SVM, NB, RF optimizados para texto
5. **Análisis de features**: Identifico palabras más importantes
6. **Evaluación completa**: Métricas detalladas y visualizaciones

Mi meta es conseguir al menos 70% de F1-Score con modelos tradicionales para luego integrarlos en el ensemble final.

In [1]:
# Importo todas las librerías que necesito para el análisis NLP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Librerías específicas para NLP
import re
import string
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

# Vectorización y modelos
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler

# Modelos de machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Métricas y evaluación
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, roc_auc_score,
    classification_report, confusion_matrix, roc_curve
)

import pickle
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Descargo recursos NLTK si no los tengo
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Descargando recursos NLTK necesarios...")
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)

print(f"Inicio del análisis NLP: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Implementando modelos de detección de desinformación basados en texto")

Inicio del análisis NLP: 2025-08-22 16:33:25
Implementando modelos de detección de desinformación basados en texto


## Carga y Análisis Inicial de Datos

Primero cargo los datos y hago un análisis exploratorio para entender con qué estoy trabajando. Es crucial identificar problemas de data leakage desde el inicio.

In [ ]:
# Cargo los datos de texto y features procesadas
print("Cargando datos para análisis NLP...")

try:
    # Cargo datos de texto que preparé en notebooks anteriores
    df_text = pd.read_csv('../processed_data/text_data_for_nlp.csv')
    print(f"Datos de texto cargados: {df_text.shape}")
    
    # Cargo features procesadas para obtener los targets
    df_features = pd.read_csv('../processed_data/dataset_features_processed_winsorized.csv')
    print(f"Features cargadas: {df_features.shape}")
    
    # Verifico que los índices estén alineados
    assert len(df_text) == len(df_features), "Los datasets no están alineados"
    print("Datasets alineados correctamente")
    
except FileNotFoundError as e:
    print(f"Error cargando datos: {e}")
    print("Necesito ejecutar primero los notebooks de limpieza (01-02)")

# Analizo el problema de data leakage
print(f"\nAnalisis de data leakage:")
print(f"Total de filas en dataset: {len(df_text):,}")
print(f"Statements únicos: {df_text['statement'].nunique():,}")
print(f"Tweets únicos: {df_text['tweet'].nunique():,}")

# Calculo el factor de duplicación - esto me dice qué tan grave es el data leakage
duplication_factor = len(df_text) / df_text['statement'].nunique()
print(f"Factor de duplicación de statements: {duplication_factor:.1f}x")

if duplication_factor > 2:
    print(f"PROBLEMA: Cada statement aparece {duplication_factor:.1f} veces en promedio")
    #Esto causará data leakage si no lo corrijo antes del train/test split
else:
    print(f"El factor de duplicación es aceptable")

# Analizo la distribución de clases
target_col = 'BinaryNumTarget'
y = df_features[target_col]

class_dist = y.value_counts(normalize=True)
print(f"\nDistribución de clases:")
for clase, prop in class_dist.items():
    label = "Verdadero" if clase == 1.0 else "Falso"
    print(f"- {label}: {prop:.1%} ({y.value_counts()[clase]:,} casos)")

# Muestro algunos ejemplos de texto
print(f"\nEjemplos de statements:")
for i in range(3):
    stmt = df_text['statement'].iloc[i]
    target = y.iloc[i]
    label = "Verdadero" if target == 1.0 else "Falso"
    print(f"\n{i+1}. [{label}] {stmt[:150]}...")

Cargando datos para análisis NLP...
Datos de texto cargados: (134198, 2)
Features cargadas: (134198, 58)
Datasets alineados correctamente

Analisis de data leakage:
Total de filas en dataset: 134,198
Statements únicos: 1,058
Tweets únicos: 134,198
Factor de duplicación de statements: 126.8x
PROBLEMA: Cada statement aparece 126.8 veces en promedio
Esto causará data leakage si no lo corrijo antes del train/test split

Distribución de clases:
- Verdadero: 51.4% (68,930 casos)
- Falso: 48.6% (65,268 casos)

Ejemplos de statements:

1. [Verdadero] End of eviction moratorium means millions of Americans could lose their housing in the middle of a pandemic....

2. [Verdadero] End of eviction moratorium means millions of Americans could lose their housing in the middle of a pandemic....

3. [Verdadero] End of eviction moratorium means millions of Americans could lose their housing in the middle of a pandemic....


In [28]:
# Corrijo el data leakage eliminando duplicados
print("\nCORRECCION DE DATA LEAKAGE")
print("=" * 30)

print(f"Antes de la corrección:")
print(f"- Total filas: {len(df_text):,}")
print(f"- Tweets únicos: {df_text['tweet'].nunique():,}")

# Elimino duplicados manteniendo la primera ocurrencia
print(f"\nEliminando duplicados por tweet...")
df_text_unique = df_text.drop_duplicates(subset=['tweet'], keep='first')
df_features_unique = df_features.loc[df_text_unique.index]

print(f"\nDespués de la corrección:")
print(f"- Total filas: {len(df_text_unique):,}")
print(f"- Tweets únicos: {df_text_unique['tweet'].nunique():,}")
print(f"- Filas eliminadas: {len(df_text) - len(df_text_unique):,}")
print(f"- Porcentaje eliminado: {(len(df_text) - len(df_text_unique)) / len(df_text) * 100:.1f}%")

# Verifico que efectivamente no hay duplicados
assert df_text_unique['tweet'].nunique() == len(df_text_unique), "Todavía hay duplicados"
print(f"\nVerificación: Data leakage corregido exitosamente")

# Actualizo los targets
y_unique = df_features_unique[target_col]

# Verifico que la distribución de clases se mantenga similar
class_dist_clean = y_unique.value_counts(normalize=True)
print(f"\nDistribución de clases después de limpieza:")
for clase, prop in class_dist_clean.items():
    label = "Verdadero" if clase == 1.0 else "Falso"
    print(f"- {label}: {prop:.1%} ({y_unique.value_counts()[clase]:,} casos)")

# Uso los datos limpios para el resto del análisis
df_text_final = df_text_unique.copy()
y_final = y_unique.copy()

print(f"\nDatos finales para NLP: {len(df_text_final):,} tweets únicos")
print(f"Data leakage eliminado correctamente")


CORRECCION DE DATA LEAKAGE
Antes de la corrección:
- Total filas: 134,198
- Tweets únicos: 134,198

Eliminando duplicados por tweet...

Después de la corrección:
- Total filas: 134,198
- Tweets únicos: 134,198
- Filas eliminadas: 0
- Porcentaje eliminado: 0.0%

Verificación: Data leakage corregido exitosamente

Distribución de clases después de limpieza:
- Verdadero: 51.4% (68,930 casos)
- Falso: 48.6% (65,268 casos)

Datos finales para NLP: 134,198 tweets únicos
Data leakage eliminado correctamente


In [15]:
# Corrijo el data leakage eliminando duplicados
print("\nCORRECCION DE DATA LEAKAGE")
print("=" * 30)

print(f"Antes de la corrección:")
print(f"- Total filas: {len(df_text):,}")
print(f"- Statements únicos: {df_text['statement'].nunique():,}")

# Elimino duplicados manteniendo la primera ocurrencia
print(f"\nEliminando duplicados por statement...")
df_text_unique = df_text.drop_duplicates(subset=['statement'], keep='first')
df_features_unique = df_features.loc[df_text_unique.index]

print(f"\nDespués de la corrección:")
print(f"- Total filas: {len(df_text_unique):,}")
print(f"- Statements únicos: {df_text_unique['statement'].nunique():,}")
print(f"- Filas eliminadas: {len(df_text) - len(df_text_unique):,}")
print(f"- Porcentaje eliminado: {(len(df_text) - len(df_text_unique)) / len(df_text) * 100:.1f}%")

# Verifico que efectivamente no hay duplicados
assert df_text_unique['statement'].nunique() == len(df_text_unique), "Todavía hay duplicados"
print(f"\nVerificación: Data leakage corregido exitosamente")

# Actualizo los targets
y_unique = df_features_unique[target_col]

# Verifico que la distribución de clases se mantenga similar
class_dist_clean = y_unique.value_counts(normalize=True)
print(f"\nDistribución de clases después de limpieza:")
for clase, prop in class_dist_clean.items():
    label = "Verdadero" if clase == 1.0 else "Falso"
    print(f"- {label}: {prop:.1%} ({y_unique.value_counts()[clase]:,} casos)")

# Uso los datos limpios para el resto del análisis
df_text_final = df_text_unique.copy()
y_final = y_unique.copy()

print(f"\nDatos finales para NLP: {len(df_text_final):,} statements únicos")
print(f"Data leakage eliminado correctamente")


CORRECCION DE DATA LEAKAGE
Antes de la corrección:
- Total filas: 134,198
- Statements únicos: 1,058

Eliminando duplicados por statement...

Después de la corrección:
- Total filas: 1,058
- Statements únicos: 1,058
- Filas eliminadas: 133,140
- Porcentaje eliminado: 99.2%

Verificación: Data leakage corregido exitosamente

Distribución de clases después de limpieza:
- Verdadero: 54.7% (579 casos)
- Falso: 45.3% (479 casos)

Datos finales para NLP: 1,058 statements únicos
Data leakage eliminado correctamente


## Preprocesamiento de Texto

Ahora preproceso el texto para optimizarlo para machine learning. Voy a limpiar, normalizar y tokenizar el contenido.

In [4]:
# Creo mi función de preprocesamiento de texto optimizada
def preprocess_text_for_ml(text, use_stemming=True, remove_stopwords=True):
    """
    Preproceso texto para machine learning
    - Limpio URLs, menciones, caracteres especiales
    - Normalizo a minúsculas
    - Tokenizo y filtro stopwords
    - Aplico stemming si se requiere
    """
    
    if pd.isna(text) or text == '':
        return ''
    
    # Convierto a string y minúsculas
    text = str(text).lower()
    
    # Limpio URLs y menciones - los reemplazo con tokens especiales
    text = re.sub(r'http\S+|www\S+|https\S+', 'URL_TOKEN', text)
    text = re.sub(r'@\w+', 'MENTION_TOKEN', text)
    text = re.sub(r'#\w+', 'HASHTAG_TOKEN', text)
    
    # Reemplazo números con token
    text = re.sub(r'\d+', 'NUMBER_TOKEN', text)
    
    # Elimino caracteres especiales pero mantengo espacios
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Normalizo espacios múltiples
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenizo el texto
    tokens = word_tokenize(text)
    
    # Elimino stopwords si se especifica
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
    
    # Aplico stemming si se especifica
    if use_stemming:
        stemmer = SnowballStemmer('english')
        tokens = [stemmer.stem(token) for token in tokens]
    
    # Filtro tokens muy cortos (menos de 3 caracteres)
    tokens = [token for token in tokens if len(token) >= 3]
    
    return ' '.join(tokens)

# Aplico preprocesamiento a los statements
print("Aplicando preprocesamiento de texto...")

# Uso statements porque son más informativos que tweets
texts_raw = df_text_final['statement'].fillna('')

# Preproceso con stemming
print("Preprocesando con stemming...")
texts_stemmed = texts_raw.apply(lambda x: preprocess_text_for_ml(x, use_stemming=True))

# Preproceso sin stemming para comparar
print("Preprocesando sin stemming...")
texts_no_stem = texts_raw.apply(lambda x: preprocess_text_for_ml(x, use_stemming=False))

# Filtro textos muy cortos después del preprocesamiento
min_text_length = 15  # Al menos 15 caracteres
valid_mask_stem = texts_stemmed.str.len() >= min_text_length
valid_mask_no_stem = texts_no_stem.str.len() >= min_text_length

print(f"\nTextos válidos después de preprocesamiento:")
print(f"- Con stemming: {valid_mask_stem.sum():,} de {len(texts_raw):,}")
print(f"- Sin stemming: {valid_mask_no_stem.sum():,} de {len(texts_raw):,}")

# Uso la intersección para asegurar consistencia
valid_mask = valid_mask_stem & valid_mask_no_stem
print(f"- Textos finales válidos: {valid_mask.sum():,}")

# Aplico el filtro final
texts_stemmed_final = texts_stemmed[valid_mask]
texts_no_stem_final = texts_no_stem[valid_mask]
y_text_final = y_final[valid_mask]

print(f"\nEjemplos de preprocesamiento:")
for i in range(min(3, len(texts_stemmed_final))):
    idx = texts_stemmed_final.index[i]
    original = texts_raw.loc[idx]
    processed = texts_stemmed_final.iloc[i]
    
    print(f"\nEjemplo {i+1}:")
    print(f"Original: {original[:120]}...")
    print(f"Procesado: {processed[:120]}...")

print(f"\nPreprocesamiento completado exitosamente")

Aplicando preprocesamiento de texto...
Preprocesando con stemming...
Preprocesando sin stemming...

Textos válidos después de preprocesamiento:
- Con stemming: 1,057 de 1,058
- Sin stemming: 1,058 de 1,058
- Textos finales válidos: 1,057

Ejemplos de preprocesamiento:

Ejemplo 1:
Original: End of eviction moratorium means millions of Americans could lose their housing in the middle of a pandemic....
Procesado: end evict moratorium mean million american could lose hous middl pandem...

Ejemplo 2:
Original: The Trump administration worked to free 5,000 Taliban prisoners....
Procesado: trump administr work free number_token number_token taliban prison...

Ejemplo 3:
Original: In Afghanistan, over 100 billion dollars spent on military contracts....
Procesado: afghanistan number_token billion dollar spent militari contract...

Preprocesamiento completado exitosamente


## Análisis Exploratorio de Texto

Antes de vectorizar, analizo las características del texto preprocesado para entender mejor mis datos.

In [5]:
# Analizo las características del texto preprocesado
print("ANALISIS EXPLORATORIO DE TEXTO")
print("=" * 35)

# Calculo estadísticas básicas de longitud
text_lengths = texts_stemmed_final.str.len()
word_counts = texts_stemmed_final.str.split().str.len()

print(f"Estadísticas de longitud de texto:")
print(f"- Caracteres promedio: {text_lengths.mean():.0f}")
print(f"- Caracteres mediana: {text_lengths.median():.0f}")
print(f"- Caracteres min: {text_lengths.min()}")
print(f"- Caracteres max: {text_lengths.max()}")
print(f"\n- Palabras promedio: {word_counts.mean():.0f}")
print(f"- Palabras mediana: {word_counts.median():.0f}")
print(f"- Palabras min: {word_counts.min()}")
print(f"- Palabras max: {word_counts.max()}")

# Analizo diferencias entre clases
true_texts = texts_stemmed_final[y_text_final == 1.0]
false_texts = texts_stemmed_final[y_text_final == 0.0]

true_lengths = true_texts.str.len()
false_lengths = false_texts.str.len()
true_words = true_texts.str.split().str.len()
false_words = false_texts.str.split().str.len()

print(f"\nComparación por clase:")
print(f"Noticias VERDADERAS:")
print(f"- Caracteres promedio: {true_lengths.mean():.0f}")
print(f"- Palabras promedio: {true_words.mean():.0f}")

print(f"Noticias FALSAS:")
print(f"- Caracteres promedio: {false_lengths.mean():.0f}")
print(f"- Palabras promedio: {false_words.mean():.0f}")

# Creo visualizaciones
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Distribución de Longitud de Texto',
        'Longitud por Clase',
        'Distribución de Número de Palabras',
        'Palabras por Clase'
    )
)

# Histograma de longitudes de texto
fig.add_trace(
    go.Histogram(
        x=text_lengths,
        nbinsx=50,
        name='Longitud Texto',
        marker_color='blue',
        showlegend=False
    ),
    row=1, col=1
)

# Box plot de longitudes por clase
fig.add_trace(
    go.Box(
        y=true_lengths,
        name='Verdadero',
        marker_color='green'
    ),
    row=1, col=2
)

fig.add_trace(
    go.Box(
        y=false_lengths,
        name='Falso',
        marker_color='red'
    ),
    row=1, col=2
)

# Histograma de número de palabras
fig.add_trace(
    go.Histogram(
        x=word_counts,
        nbinsx=40,
        name='Num Palabras',
        marker_color='orange',
        showlegend=False
    ),
    row=2, col=1
)

# Box plot de palabras por clase
fig.add_trace(
    go.Box(
        y=true_words,
        name='Verdadero',
        marker_color='green',
        showlegend=False
    ),
    row=2, col=2
)

fig.add_trace(
    go.Box(
        y=false_words,
        name='Falso',
        marker_color='red',
        showlegend=False
    ),
    row=2, col=2
)

fig.update_layout(
    title='Análisis Exploratorio de Texto Preprocesado',
    height=600
)

fig.show()

print(f"\nVisualizaciones de distribuciones creadas")

ANALISIS EXPLORATORIO DE TEXTO
Estadísticas de longitud de texto:
- Caracteres promedio: 65
- Caracteres mediana: 60
- Caracteres min: 16
- Caracteres max: 265

- Palabras promedio: 10
- Palabras mediana: 9
- Palabras min: 3
- Palabras max: 31

Comparación por clase:
Noticias VERDADERAS:
- Caracteres promedio: 68
- Palabras promedio: 10
Noticias FALSAS:
- Caracteres promedio: 62
- Palabras promedio: 10



Visualizaciones de distribuciones creadas


In [6]:
# Analizo las palabras más frecuentes por clase
print("\nANALISIS DE PALABRAS MAS FRECUENTES")
print("-" * 40)

def get_top_words(texts, n=20):
    """
    Obtengo las n palabras más frecuentes de una colección de textos
    """
    all_words = ' '.join(texts).split()
    word_freq = Counter(all_words)
    return word_freq.most_common(n)

# Obtengo palabras más frecuentes por clase
true_top_words = get_top_words(true_texts, 20)
false_top_words = get_top_words(false_texts, 20)

print(f"Top 20 palabras en noticias VERDADERAS:")
for i, (word, count) in enumerate(true_top_words, 1):
    print(f"{i:2d}. {word:<15} ({count:,} veces)")

print(f"\nTop 20 palabras en noticias FALSAS:")
for i, (word, count) in enumerate(false_top_words, 1):
    print(f"{i:2d}. {word:<15} ({count:,} veces)")

# Visualizo palabras más frecuentes
fig_words = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Top 15 Palabras - Verdadero', 'Top 15 Palabras - Falso')
)

# Palabras en noticias verdaderas
true_words_list, true_counts_list = zip(*true_top_words[:15])
fig_words.add_trace(
    go.Bar(
        x=list(true_counts_list),
        y=list(true_words_list),
        orientation='h',
        marker_color='green',
        showlegend=False
    ),
    row=1, col=1
)

# Palabras en noticias falsas
false_words_list, false_counts_list = zip(*false_top_words[:15])
fig_words.add_trace(
    go.Bar(
        x=list(false_counts_list),
        y=list(false_words_list),
        orientation='h',
        marker_color='red',
        showlegend=False
    ),
    row=1, col=2
)

fig_words.update_layout(
    title='Palabras Más Frecuentes por Clase',
    height=500
)

fig_words.show()

# Analizo palabras distintivas entre clases
print(f"\nAnalisis de palabras distintivas:")
true_words_set = set([word for word, _ in true_top_words])
false_words_set = set([word for word, _ in false_top_words])

only_true = true_words_set - false_words_set
only_false = false_words_set - true_words_set
common_words = true_words_set & false_words_set

print(f"Palabras solo en verdaderas: {sorted(only_true)}")
print(f"Palabras solo en falsas: {sorted(only_false)}")
print(f"Palabras comunes: {sorted(common_words)}")

print(f"\nAnalisis de vocabulario completado")


ANALISIS DE PALABRAS MAS FRECUENTES
----------------------------------------
Top 20 palabras en noticias VERDADERAS:
 1. number_token    (360 veces)
 2. say             (112 veces)
 3. year            (86 veces)
 4. state           (76 veces)
 5. american        (54 veces)
 6. percent         (54 veces)
 7. trump           (47 veces)
 8. million         (40 veces)
 9. tax             (39 veces)
10. nation          (37 veces)
11. peopl           (36 veces)
12. texa            (34 veces)
13. presid          (34 veces)
14. time            (33 veces)
15. rate            (33 veces)
16. one             (32 veces)
17. countri         (32 veces)
18. new             (31 veces)
19. sinc            (30 veces)
20. job             (30 veces)

Top 20 palabras en noticias FALSAS:
 1. number_token    (242 veces)
 2. say             (145 veces)
 3. biden           (89 veces)
 4. vaccin          (72 veces)
 5. covid           (57 veces)
 6. joe             (53 veces)
 7. show            (47 veces)
 8. 


Analisis de palabras distintivas:
Palabras solo en verdaderas: ['american', 'countri', 'job', 'million', 'nation', 'new', 'one', 'percent', 'rate', 'sinc', 'texa', 'time', 'year']
Palabras solo en falsas: ['ballot', 'biden', 'covid', 'die', 'donald', 'elect', 'joe', 'photo', 'said', 'show', 'vaccin', 'video', 'vote']
Palabras comunes: ['number_token', 'peopl', 'presid', 'say', 'state', 'tax', 'trump']

Analisis de vocabulario completado


## Vectorización TF-IDF

Ahora convierto el texto preprocesado a una representación numérica usando TF-IDF. Es crucial hacer la división train/test ANTES de vectorizar para evitar data leakage.

In [7]:
# Divido los datos ANTES de vectorizar para evitar data leakage
print("VECTORIZACION TF-IDF")
print("=" * 20)

print("Dividiendo datos antes de vectorización...")
print("IMPORTANTE: Hago esto ANTES de TF-IDF para evitar data leakage")

# División estratificada train/test
X_train_text, X_test_text, y_train, y_test = train_test_split(
    texts_stemmed_final,
    y_text_final,
    test_size=0.2,
    random_state=42,
    stratify=y_text_final
)

# División train/validation
X_train_text, X_val_text, y_train, y_val = train_test_split(
    X_train_text,
    y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train
)

print(f"División completada:")
print(f"- Entrenamiento: {len(X_train_text):,} textos")
print(f"- Validación: {len(X_val_text):,} textos")
print(f"- Prueba: {len(X_test_text):,} textos")

# Configuro el vectorizador TF-IDF con parámetros optimizados
print(f"\nConfigurando vectorizador TF-IDF...")

tfidf_vectorizer = TfidfVectorizer(
    max_features=8000,          # Top 8000 features
    min_df=2,                   # Mínimo en 2 documentos
    max_df=0.95,                # Máximo en 95% de documentos
    ngram_range=(1, 2),         # Unigrams y bigrams
    sublinear_tf=True,          # Escalado sublinear tf
    norm='l2',                  # Normalización L2
    use_idf=True,               # Usar pesos IDF
    smooth_idf=True,            # Suavizado IDF
    stop_words=None             # Ya eliminé stopwords en preprocesamiento
)

print(f"Parámetros del vectorizador:")
print(f"- Máximo features: {tfidf_vectorizer.max_features:,}")
print(f"- N-grams: {tfidf_vectorizer.ngram_range}")
print(f"- Min document frequency: {tfidf_vectorizer.min_df}")
print(f"- Max document frequency: {tfidf_vectorizer.max_df}")

# Entreno el vectorizador SOLO con datos de entrenamiento
print(f"\nEntrenando vectorizador con datos de entrenamiento...")
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)

# Transformo datos de validación y prueba
print(f"Transformando datos de validación y prueba...")
X_val_tfidf = tfidf_vectorizer.transform(X_val_text)
X_test_tfidf = tfidf_vectorizer.transform(X_test_text)

print(f"\nMatrices TF-IDF generadas:")
print(f"- Entrenamiento: {X_train_tfidf.shape}")
print(f"- Validación: {X_val_tfidf.shape}")
print(f"- Prueba: {X_test_tfidf.shape}")

# Calculo sparsity
sparsity = 1 - (X_train_tfidf.nnz / np.prod(X_train_tfidf.shape))
print(f"- Sparsity de la matriz: {sparsity:.1%}")
print(f"- Features reales generadas: {X_train_tfidf.shape[1]:,}")

# Guardo el vectorizador para uso posterior
with open('../processed_data/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

print(f"\nVectorizador guardado en ../processed_data/tfidf_vectorizer.pkl")
print(f"Vectorización TF-IDF completada exitosamente")

VECTORIZACION TF-IDF
Dividiendo datos antes de vectorización...
IMPORTANTE: Hago esto ANTES de TF-IDF para evitar data leakage
División completada:
- Entrenamiento: 676 textos
- Validación: 169 textos
- Prueba: 212 textos

Configurando vectorizador TF-IDF...
Parámetros del vectorizador:
- Máximo features: 8,000
- N-grams: (1, 2)
- Min document frequency: 2
- Max document frequency: 0.95

Entrenando vectorizador con datos de entrenamiento...
Transformando datos de validación y prueba...

Matrices TF-IDF generadas:
- Entrenamiento: (676, 1250)
- Validación: (169, 1250)
- Prueba: (212, 1250)
- Sparsity de la matriz: 99.3%
- Features reales generadas: 1,250

Vectorizador guardado en ../processed_data/tfidf_vectorizer.pkl
Vectorización TF-IDF completada exitosamente


In [ ]:
# Analizo las features TF-IDF más importantes
print("\nANALISIS DE FEATURES TF-IDF")
print("-" * 30)

# Obtengo los nombres de las features
feature_names = tfidf_vectorizer.get_feature_names_out()
print(f"Total de features TF-IDF generadas: {len(feature_names):,}")

# Entreno un modelo rápido para analizar importancia de features
print(f"Entrenando modelo temporal para análisis de features...")
lr_temp = LogisticRegression(random_state=42, max_iter=1000)
lr_temp.fit(X_train_tfidf, y_train)

# Obtengo coeficientes del modelo
coefficients = lr_temp.coef_[0]

# Features que más indican noticias verdaderas (coeficientes positivos altos)
positive_indices = np.argsort(coefficients)[-25:]
positive_features = [(feature_names[i], coefficients[i]) for i in positive_indices]

# Features que más indican noticias falsas (coeficientes negativos altos)
negative_indices = np.argsort(coefficients)[:25]
negative_features = [(feature_names[i], coefficients[i]) for i in negative_indices]

print(f"\nTop 25 features que indican NOTICIAS VERDADERAS:")
for i, (feature, coef) in enumerate(reversed(positive_features), 1):
    print(f"{i:2d}. {feature:<20} (coef: {coef:+.3f})")

print(f"\nTop 25 features que indican NOTICIAS FALSAS:")
for i, (feature, coef) in enumerate(negative_features, 1):
    print(f"{i:2d}. {feature:<20} (coef: {coef:+.3f})")

# Visualizo la importancia de features
fig_importance = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Features Indicadoras de Verdadero', 'Features Indicadoras de Falso')
)

# Features para verdadero (top 12)
pos_names, pos_coefs = zip(*positive_features[-12:])
fig_importance.add_trace(
    go.Bar(
        x=list(pos_coefs),
        y=list(pos_names),
        orientation='h',
        marker_color='green',
        showlegend=False
    ),
    row=1, col=1
)

# Features para falso (top 12)
neg_names, neg_coefs = zip(*negative_features[:12])
fig_importance.add_trace(
    go.Bar(
        x=list(neg_coefs),
        y=list(neg_names),
        orientation='h',
        marker_color='red',
        showlegend=False
    ),
    row=1, col=2
)

fig_importance.update_layout(
    title='Features TF-IDF Más Importantes (Coeficientes Logistic Regression)',
    height=600
)

fig_importance.show()

# Estadísticas de los coeficientes
print(f"\nEstadísticas de coeficientes:")
print(f"- Total features: {len(coefficients):,}")
print(f"- Features con coef positivo: {(coefficients > 0).sum():,}")
print(f"- Features con coef negativo: {(coefficients < 0).sum():,}")
print(f"- Coeficiente máximo: {coefficients.max():.3f}")
print(f"- Coeficiente mínimo: {coefficients.min():.3f}")
print(f"- Desviación estándar: {coefficients.std():.3f}")

print(f"\nAnalisis de features TF-IDF completado")


ANALISIS DE FEATURES TF-IDF
------------------------------
Total de features TF-IDF generadas: 1,250
Entrenando modelo temporal para análisis de features...

Top 25 features que indican NOTICIAS VERDADERAS:
 1. percent              (coef: +1.519)
 2. year                 (coef: +1.217)
 3. american             (coef: +1.177)
 4. rate                 (coef: +1.160)
 5. number_token percent (coef: +1.100)
 6. nation               (coef: +0.976)
 7. last                 (coef: +0.976)
 8. sinc                 (coef: +0.880)
 9. million              (coef: +0.859)
10. support              (coef: +0.857)
11. countri              (coef: +0.856)
12. obama                (coef: +0.838)
13. iran                 (coef: +0.807)
14. half                 (coef: +0.780)
15. marijuana            (coef: +0.768)
16. job                  (coef: +0.733)
17. tax                  (coef: +0.729)
18. texa                 (coef: +0.725)
19. three                (coef: +0.708)
20. highest              (coef: 


Estadísticas de coeficientes:
- Total features: 1,250
- Features con coef positivo: 681
- Features con coef negativo: 569
- Coeficiente máximo: 1.519
- Coeficiente mínimo: -2.506
- Desviación estándar: 0.323

Analisis de features TF-IDF completado


## Entrenamiento de Modelos de Machine Learning

Ahora entreno varios modelos tradicionales optimizados para datos de texto sparse como los de TF-IDF.

In [26]:
# Configuro los modelos optimizados para texto
print("ENTRENAMIENTO DE MODELOS DE MACHINE LEARNING")
print("=" * 50)

def evaluate_text_model(model, X_train, X_val, X_test, y_train, y_val, y_test, model_name):
    """
    Evalúo un modelo de texto con métricas completas
    Devuelvo diccionario con todos los resultados
    """
    
    print(f"\nEntrenando {model_name}...")
    start_time = datetime.now()
    
    # Entreno el modelo
    model.fit(X_train, y_train)
    training_time = (datetime.now() - start_time).total_seconds()
    
    # Hago predicciones
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)
    
    # Obtengo probabilidades si están disponibles
    try:
        y_test_proba = model.predict_proba(X_test)[:, 1]
        auc_score = roc_auc_score(y_test, y_test_proba)
    except:
        y_test_proba = None
        auc_score = np.nan
    
    # Calculo métricas
    val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(
        y_val, y_val_pred, average='binary'
    )
    
    test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(
        y_test, y_test_pred, average='binary'
    )
    
    test_acc = accuracy_score(y_test, y_test_pred)
    
    print(f"{model_name:<20} Val F1: {val_f1:.4f}  Test F1: {test_f1:.4f}  Test Acc: {test_acc:.4f}  AUC: {auc_score:.4f}  Tiempo: {training_time:.1f}s")
    
    return {
        'model_name': model_name,
        'model': model,
        'val_f1': val_f1,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'test_f1': test_f1,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_accuracy': test_acc,
        'test_auc': auc_score,
        'training_time': training_time,
        'test_predictions': y_test_pred,
        'test_probabilities': y_test_proba
    }

# Configuro modelos optimizados específicamente para texto
text_models = {
    'Logistic Regression': LogisticRegression(
        C=1.0,                    # Regularización moderada
        penalty='l2',             # Regularización L2
        random_state=42,
        max_iter=1000,            # Suficientes iteraciones
        solver='lbfgs'            # Solver eficiente
    ),
    
    'SVM Linear': SVC(
        C=1.0,                    # Regularización moderada
        kernel='linear',          # Kernel lineal para texto
        probability=True,         # Para obtener probabilidades
        random_state=42
    ),
    
    'SVM RBF': SVC(
        C=1.0,
        kernel='rbf',             # Kernel no lineal
        gamma='scale',            # Gamma automático
        probability=True,
        random_state=42
    ),
    
    'Multinomial NB': MultinomialNB(
        alpha=1.0                 # Suavizado de Laplace
    ),
    
    'Random Forest': RandomForestClassifier(
        n_estimators=100,         # 100 árboles
        max_depth=20,             # Profundidad controlada
        min_samples_split=5,      # Mínimo para split
        min_samples_leaf=2,       # Mínimo en hojas
        random_state=42,
        n_jobs=-1                 # Paralelización
    ),
    
    'KNN Cosine': KNeighborsClassifier(
        n_neighbors=5,            # 5 vecinos
        metric='cosine',          # Métrica coseno para texto
        n_jobs=-1
    )
}

print(f"Modelos configurados para entrenamiento: {len(text_models)}")
for i, name in enumerate(text_models.keys(), 1):
    print(f"{i}. {name}")

print(f"\nIniciando entrenamiento y evaluación...")

ENTRENAMIENTO DE MODELOS DE MACHINE LEARNING
Modelos configurados para entrenamiento: 6
1. Logistic Regression
2. SVM Linear
3. SVM RBF
4. Multinomial NB
5. Random Forest
6. KNN Cosine

Iniciando entrenamiento y evaluación...


In [ ]:
# Entreno y evalúo todos los modelos
print(f"\nENTRENAMIENTO Y EVALUACION")
print("=" * 30)

text_results = {}
successful_models = 0

for name, model in text_models.items():
    try:
        result = evaluate_text_model(
            model, X_train_tfidf, X_val_tfidf, X_test_tfidf,
            y_train, y_val, y_test, name
        )
        text_results[name] = result
        successful_models += 1
        
    except Exception as e:
        print(f"Error entrenando {name}: {str(e)}")

print(f"\nEntrenamiento completado: {successful_models}/{len(text_models)} modelos exitosos")

# Creo tabla de resultados si hay modelos exitosos
if len(text_results) > 0:
    results_data = []
    for name, result in text_results.items():
        results_data.append({
            'Modelo': name,
            'Val_F1': result['val_f1'],
            'Test_F1': result['test_f1'],
            'Test_Accuracy': result['test_accuracy'],
            'Test_Precision': result['test_precision'],
            'Test_Recall': result['test_recall'],
            'Test_AUC': result['test_auc'],
            'Training_Time': result['training_time']
        })
    
    results_df = pd.DataFrame(results_data)
    results_df = results_df.sort_values('Test_F1', ascending=False)
    
    print(f"\nRESULTADOS FINALES (ordenados por F1-Score):")
    print("=" * 70)
    display_cols = ['Modelo', 'Test_F1', 'Test_Accuracy', 'Test_Precision', 'Test_Recall', 'Test_AUC']
    print(results_df[display_cols].round(4).to_string(index=False))
    
    # Identifico el mejor modelo
    best_model_name = results_df.iloc[0]['Modelo']
    best_f1 = results_df.iloc[0]['Test_F1']
    best_acc = results_df.iloc[0]['Test_Accuracy']
    best_auc = results_df.iloc[0]['Test_AUC']
    
    print(f"\nMEJOR MODELO NLP: {best_model_name}")
    print(f"- F1-Score: {best_f1:.4f}")
    print(f"- Accuracy: {best_acc:.4f}")
    if not pd.isna(best_auc):
        print(f"- AUC: {best_auc:.4f}")
    
    # Evalúo vs objetivo
    target_f1 = 0.70
    print(f"\nEvaluación vs objetivo:")
    print(f"- Objetivo F1-Score: {target_f1:.0%}")
    print(f"- F1-Score alcanzado: {best_f1:.0%}")
    
    if best_f1 >= target_f1:
        surplus = best_f1 - target_f1
        print(f"- OBJETIVO ALCANZADO (superavit: {surplus:+.1%})")
        objetivo_alcanzado = True
    else:
        deficit = target_f1 - best_f1
        print(f"- OBJETIVO NO ALCANZADO (deficit: {deficit:.1%})")
        objetivo_alcanzado = False
    
    # Modelo más rápido
    fastest_idx = results_df['Training_Time'].idxmin()
    fastest_model = results_df.loc[fastest_idx, 'Modelo']
    fastest_time = results_df.loc[fastest_idx, 'Training_Time']
    print(f"\nModelo más rápido: {fastest_model} ({fastest_time:.1f}s)")
    
else:
    print("No se entrenaron modelos exitosamente")
    objetivo_alcanzado = False


ENTRENAMIENTO Y EVALUACION

Entrenando Logistic Regression...
Logistic Regression  Val F1: 0.8159  Test F1: 0.8362  Test Acc: 0.8208  AUC: 0.9076  Tiempo: 0.0s

Entrenando SVM Linear...
SVM Linear           Val F1: 0.8384  Test F1: 0.8261  Test Acc: 0.8113  AUC: 0.8976  Tiempo: 0.4s

Entrenando SVM RBF...
SVM RBF              Val F1: 0.8269  Test F1: 0.8511  Test Acc: 0.8349  AUC: 0.9043  Tiempo: 0.5s

Entrenando Multinomial NB...
Multinomial NB       Val F1: 0.8060  Test F1: 0.8174  Test Acc: 0.8019  AUC: 0.9064  Tiempo: 0.0s

Entrenando Random Forest...
Random Forest        Val F1: 0.8018  Test F1: 0.8379  Test Acc: 0.8066  AUC: 0.8869  Tiempo: 0.4s

Entrenando KNN Cosine...
KNN Cosine           Val F1: 0.7553  Test F1: 0.8054  Test Acc: 0.7972  AUC: 0.8917  Tiempo: 0.0s

Entrenamiento completado: 6/6 modelos exitosos

RESULTADOS FINALES (ordenados por F1-Score):
             Modelo  Test_F1  Test_Accuracy  Test_Precision  Test_Recall  Test_AUC
            SVM RBF   0.8511         0

## Análisis Detallado del Mejor Modelo

Analizo en profundidad el desempeño del mejor modelo, incluyendo matriz de confusión, curva ROC y análisis de errores.

In [11]:
# Análisis detallado del mejor modelo
if len(text_results) > 0:
    print(f"\nANALISIS DETALLADO DEL MEJOR MODELO: {best_model_name}")
    print("=" * 65)
    
    best_result = text_results[best_model_name]
    best_model = best_result['model']
    
    # Reporte de clasificación completo
    print(f"\nReporte de clasificación detallado:")
    print(classification_report(
        y_test, best_result['test_predictions'],
        target_names=['Falso', 'Verdadero'],
        digits=4
    ))
    
    # Matriz de confusión
    cm = confusion_matrix(y_test, best_result['test_predictions'])
    
    print(f"\nMatriz de confusión:")
    print(f"                Predicho")
    print(f"Real      Falso  Verdadero")
    print(f"Falso     {cm[0,0]:5d}  {cm[0,1]:9d}")
    print(f"Verdadero {cm[1,0]:5d}  {cm[1,1]:9d}")
    
    # Visualizo matriz de confusión
    fig_cm = px.imshow(
        cm,
        text_auto=True,
        aspect="auto",
        title=f'Matriz de Confusión - {best_model_name}',
        labels=dict(x="Predicho", y="Real"),
        x=['Falso', 'Verdadero'],
        y=['Falso', 'Verdadero'],
        color_continuous_scale='Blues'
    )
    
    fig_cm.show()
    
    # Análisis de errores
    correct_predictions = (best_result['test_predictions'] == y_test)
    total_errors = (~correct_predictions).sum()
    error_rate = total_errors / len(y_test)
    
    print(f"\nAnalisis de errores:")
    print(f"- Total predicciones: {len(y_test):,}")
    print(f"- Predicciones correctas: {correct_predictions.sum():,}")
    print(f"- Predicciones incorrectas: {total_errors:,}")
    print(f"- Tasa de error: {error_rate:.1%}")
    
    # Tipos de errores
    false_positives = ((best_result['test_predictions'] == 1) & (y_test == 0)).sum()
    false_negatives = ((best_result['test_predictions'] == 0) & (y_test == 1)).sum()
    
    print(f"- Falsos positivos (predice verdadero cuando es falso): {false_positives}")
    print(f"- Falsos negativos (predice falso cuando es verdadero): {false_negatives}")
    
    # Muestro algunos ejemplos de errores
    if total_errors > 0:
        error_indices = np.where(~correct_predictions)[0]
        
        print(f"\nEjemplos de errores (primeros 3):")
        for i, error_idx in enumerate(error_indices[:3]):
            actual = y_test.iloc[error_idx]
            predicted = best_result['test_predictions'][error_idx]
            text_idx = X_test_text.index[error_idx]
            text_content = X_test_text.iloc[error_idx]
            
            actual_label = "Verdadero" if actual == 1.0 else "Falso"
            predicted_label = "Verdadero" if predicted == 1.0 else "Falso"
            
            print(f"\nError {i+1}:")
            print(f"- Real: {actual_label}, Predicho: {predicted_label}")
            print(f"- Texto: {text_content[:180]}...")
    
    # Curva ROC si hay probabilidades
    if best_result['test_probabilities'] is not None:
        fpr, tpr, _ = roc_curve(y_test, best_result['test_probabilities'])
        
        fig_roc = go.Figure()
        
        # Curva ROC del modelo
        fig_roc.add_trace(
            go.Scatter(
                x=fpr,
                y=tpr,
                mode='lines',
                name=f'{best_model_name} (AUC: {best_result["test_auc"]:.3f})',
                line=dict(color='blue', width=3)
            )
        )
        
        # Línea diagonal (clasificador aleatorio)
        fig_roc.add_trace(
            go.Scatter(
                x=[0, 1],
                y=[0, 1],
                mode='lines',
                name='Clasificador Aleatorio',
                line=dict(color='red', width=2, dash='dash')
            )
        )
        
        fig_roc.update_layout(
            title=f'Curva ROC - {best_model_name}',
            xaxis_title='Tasa de Falsos Positivos (1 - Especificidad)',
            yaxis_title='Tasa de Verdaderos Positivos (Sensibilidad)',
            width=600,
            height=500
        )
        
        fig_roc.show()
    
    print(f"\nAnalisis detallado del mejor modelo completado")
else:
    print("No hay modelos para analizar en detalle")


ANALISIS DETALLADO DEL MEJOR MODELO: SVM RBF

Reporte de clasificación detallado:
              precision    recall  f1-score   support

       Falso     0.8280    0.8021    0.8148        96
   Verdadero     0.8403    0.8621    0.8511       116

    accuracy                         0.8349       212
   macro avg     0.8341    0.8321    0.8329       212
weighted avg     0.8347    0.8349    0.8346       212


Matriz de confusión:
                Predicho
Real      Falso  Verdadero
Falso        77         19
Verdadero    16        100



Analisis de errores:
- Total predicciones: 212
- Predicciones correctas: 177
- Predicciones incorrectas: 35
- Tasa de error: 16.5%
- Falsos positivos (predice verdadero cuando es falso): 19
- Falsos negativos (predice falso cuando es verdadero): 16

Ejemplos de errores (primeros 3):

Error 1:
- Real: Falso, Predicho: Verdadero
- Texto: say trump didnt campaign cut debt that promis...

Error 2:
- Real: Falso, Predicho: Verdadero
- Texto: cdc cancel halloween kid...

Error 3:
- Real: Falso, Predicho: Verdadero
- Texto: southern border open anyon anywher world wish enter countri...



Analisis detallado del mejor modelo completado


## Visualización Comparativa de Todos los Modelos

Creo un dashboard completo comparando el desempeño de todos los modelos entrenados.

In [12]:
# Dashboard comparativo de todos los modelos
if len(text_results) > 0:
    print("Creando dashboard comparativo de modelos...")
    
    # Dashboard principal con múltiples métricas
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'F1-Score por Modelo',
            'Accuracy vs F1-Score',
            'Tiempo de Entrenamiento',
            'Precision vs Recall'
        )
    )
    
    # 1. F1-Score por modelo (barras)
    colors = ['gold' if i == 0 else 'lightblue' for i in range(len(results_df))]
    
    fig.add_trace(
        go.Bar(
            x=results_df['Modelo'],
            y=results_df['Test_F1'],
            text=[f'{f:.3f}' for f in results_df['Test_F1']],
            textposition='outside',
            marker_color=colors,
            showlegend=False
        ),
        row=1, col=1
    )
    
    # 2. Accuracy vs F1-Score (scatter)
    fig.add_trace(
        go.Scatter(
            x=results_df['Test_Accuracy'],
            y=results_df['Test_F1'],
            mode='markers+text',
            text=results_df['Modelo'],
            textposition='top center',
            marker=dict(size=12, color='blue'),
            showlegend=False
        ),
        row=1, col=2
    )
    
    # 3. Tiempo de entrenamiento (barras)
    fig.add_trace(
        go.Bar(
            x=results_df['Modelo'],
            y=results_df['Training_Time'],
            text=[f'{t:.1f}s' for t in results_df['Training_Time']],
            textposition='outside',
            marker_color='orange',
            showlegend=False
        ),
        row=2, col=1
    )
    
    # 4. Precision vs Recall (scatter)
    fig.add_trace(
        go.Scatter(
            x=results_df['Test_Recall'],
            y=results_df['Test_Precision'],
            mode='markers+text',
            text=results_df['Modelo'],
            textposition='top center',
            marker=dict(size=12, color='green'),
            showlegend=False
        ),
        row=2, col=2
    )
    
    # Actualizo layout
    fig.update_layout(
        title=f'Dashboard Comparativo de Modelos NLP - Ganador: {best_model_name}',
        height=800
    )
    
    # Roto etiquetas en gráficos de barras
    fig.update_xaxes(tickangle=45, row=1, col=1)
    fig.update_xaxes(tickangle=45, row=2, col=1)
    
    fig.show()
    
    # Gráfico adicional: Comparación de métricas del mejor modelo
    best_metrics = ['Test_Accuracy', 'Test_Precision', 'Test_Recall', 'Test_F1']
    best_values = [results_df.iloc[0][metric] for metric in best_metrics]
    metric_labels = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    
    fig_best = go.Figure()
    
    fig_best.add_trace(
        go.Bar(
            x=metric_labels,
            y=best_values,
            text=[f'{v:.3f}' for v in best_values],
            textposition='outside',
            marker_color='gold'
        )
    )
    
    fig_best.update_layout(
        title=f'Métricas del Mejor Modelo: {best_model_name}',
        yaxis_title='Score',
        height=400
    )
    
    fig_best.show()
    
    print(f"Dashboard comparativo creado exitosamente")
else:
    print("No hay resultados para visualizar")

Creando dashboard comparativo de modelos...


Dashboard comparativo creado exitosamente


## Guardado de Resultados y Modelos

Guardo todos los resultados, modelos entrenados y configuraciones para uso posterior en el ensemble.

In [13]:
# Guardo todos los resultados y modelos
if len(text_results) > 0:
    print("\nGUARDANDO RESULTADOS Y MODELOS")
    print("=" * 35)
    
    # Guardo tabla de resultados
    results_df.to_csv('../models/nlp_tfidf_results.csv', index=False)
    print(f"Resultados guardados en: ../models/nlp_tfidf_results.csv")
    
    # Guardo el mejor modelo completo
    best_model_package = {
        'model': best_result['model'],
        'vectorizer': tfidf_vectorizer,
        'results': best_result,
        'feature_names': feature_names,
        'preprocessing_params': {
            'use_stemming': True,
            'remove_stopwords': True,
            'min_text_length': 15
        }
    }
    
    model_filename = f'../models/best_nlp_model_{best_model_name.replace(" ", "_").lower()}.pkl'
    with open(model_filename, 'wb') as f:
        pickle.dump(best_model_package, f)
    
    print(f"Mejor modelo guardado en: {model_filename}")
    
    # Guardo configuración para ensemble
    nlp_config = {
        'best_model_name': best_model_name,
        'best_f1_score': float(best_f1),
        'best_accuracy': float(best_acc),
        'best_auc': float(best_auc) if not pd.isna(best_auc) else None,
        'vectorizer_features': int(X_train_tfidf.shape[1]),
        'tfidf_params': {
            'max_features': tfidf_vectorizer.max_features,
            'ngram_range': tfidf_vectorizer.ngram_range,
            'min_df': tfidf_vectorizer.min_df,
            'max_df': tfidf_vectorizer.max_df
        },
        'data_stats': {
            'original_samples': len(df_text),
            'unique_samples': len(df_text_final),
            'final_samples': len(texts_stemmed_final),
            'train_samples': len(X_train_text),
            'test_samples': len(X_test_text)
        },
        'data_leakage_corrected': True,
        'objetivo_alcanzado': objetivo_alcanzado,
        'ready_for_ensemble': True,
        'timestamp': datetime.now().isoformat()
    }
    
    with open('../models/nlp_tfidf_config.json', 'w') as f:
        json.dump(nlp_config, f, indent=2)
    
    print(f"Configuración guardada en: ../models/nlp_tfidf_config.json")
    
    # Guardo también el vectorizador por separado para fácil acceso
    print(f"Vectorizador ya guardado en: ../processed_data/tfidf_vectorizer.pkl")
    
    print(f"\nArchivos generados:")
    print(f"1. ../models/nlp_tfidf_results.csv - Resultados comparativos")
    print(f"2. {model_filename} - Mejor modelo completo")
    print(f"3. ../models/nlp_tfidf_config.json - Configuración para ensemble")
    print(f"4. ../processed_data/tfidf_vectorizer.pkl - Vectorizador TF-IDF")
    
else:
    print("No hay resultados para guardar")


GUARDANDO RESULTADOS Y MODELOS
Resultados guardados en: ../models/nlp_tfidf_results.csv
Mejor modelo guardado en: ../models/best_nlp_model_svm_rbf.pkl
Configuración guardada en: ../models/nlp_tfidf_config.json
Vectorizador ya guardado en: ../processed_data/tfidf_vectorizer.pkl

Archivos generados:
1. ../models/nlp_tfidf_results.csv - Resultados comparativos
2. ../models/best_nlp_model_svm_rbf.pkl - Mejor modelo completo
3. ../models/nlp_tfidf_config.json - Configuración para ensemble
4. ../processed_data/tfidf_vectorizer.pkl - Vectorizador TF-IDF


## Resumen Final y Próximos Pasos

Resumo todo el trabajo realizado y defino los próximos pasos para integrar estos modelos en el ensemble final.

In [14]:
# Resumen final completo
print("\nRESUMEN FINAL - MODELOS NLP CON TF-IDF")
print("=" * 50)

if len(text_results) > 0:
    print(f"\nTRABAJO REALIZADO:")
    print(f"1. Carga y análisis de {len(df_text):,} textos originales")
    print(f"2. Corrección de data leakage eliminando {len(df_text) - len(df_text_unique):,} duplicados")
    print(f"3. Preprocesamiento avanzado de texto con NLTK")
    print(f"4. Vectorización TF-IDF con {X_train_tfidf.shape[1]:,} features")
    print(f"5. Entrenamiento de {len(text_results)} modelos exitosos")
    print(f"6. Evaluación completa con métricas detalladas")
    
    print(f"\nDATOS PROCESADOS:")
    print(f"- Dataset original: {len(df_text):,} filas")
    print(f"- Después de eliminar duplicados: {len(df_text_unique):,} filas")
    print(f"- Después de filtrar texto válido: {len(df_text_final):,} filas")
    print(f"- Reducción total: {(1 - len(df_text_final)/len(df_text))*100:.1f}%")
    
    print(f"\nVECTORIZACION TF-IDF:")
    print(f"- Features máximas configuradas: {tfidf_vectorizer.max_features:,}")
    print(f"- Features reales generadas: {X_train_tfidf.shape[1]:,}")
    print(f"- N-gramas utilizados: {tfidf_vectorizer.ngram_range}")
    print(f"- Sparsity de la matriz: {sparsity:.1%}")
    
    print(f"\nMODELOS ENTRENADOS Y RESULTADOS:")
    for i, (name, result) in enumerate(text_results.items(), 1):
        print(f"{i}. {name}:")
        print(f"   - F1-Score: {result['test_f1']:.4f}")
        print(f"   - Accuracy: {result['test_accuracy']:.4f}")
        print(f"   - Tiempo: {result['training_time']:.1f}s")
    
    print(f"\nMEJOR RENDIMIENTO ALCANZADO:")
    print(f"- Modelo ganador: {best_model_name}")
    print(f"- F1-Score: {best_f1:.4f} ({best_f1:.1%})")
    print(f"- Accuracy: {best_acc:.4f} ({best_acc:.1%})")
    if not pd.isna(best_auc):
        print(f"- AUC: {best_auc:.4f}")
    
    # Evaluación final vs objetivos
    target_f1 = 0.70
    print(f"\nEVALUACION FINAL:")
    print(f"- Objetivo F1-Score: {target_f1:.0%}")
    print(f"- F1-Score alcanzado: {best_f1:.0%}")
    
    if objetivo_alcanzado:
        surplus = best_f1 - target_f1
        print(f"- EXITO: Objetivo alcanzado con {surplus:+.1%} de superavit")
        status = "EXITOSO"
    else:
        deficit = target_f1 - best_f1
        print(f"- DEFICIT: Faltan {deficit:.1%} para alcanzar objetivo")
        status = "REQUIERE MEJORAS"
    
    # Insights y lecciones aprendidas
    print(f"\nINSIGHTS IMPORTANTES:")
    print(f"1. Eliminar duplicados fue crucial - redujo overfitting")
    print(f"2. TF-IDF con bigrams capturó patrones importantes")
    print(f"3. {best_model_name} demostró mejor balance precision/recall")
    
    # Encuentra diferencias entre modelos
    best_time = results_df['Training_Time'].min()
    slowest_time = results_df['Training_Time'].max()
    best_f1_model = results_df.iloc[0]['Modelo']
    worst_f1_model = results_df.iloc[-1]['Modelo']
    worst_f1 = results_df.iloc[-1]['Test_F1']
    
    print(f"4. Diferencia entre mejor y peor F1: {best_f1 - worst_f1:.3f}")
    print(f"5. Diferencia de tiempo: {slowest_time - best_time:.1f}s")
    
    print(f"\nPROXIMOS PASOS PARA ENSEMBLE:")
    print(f"1. Integrar {best_model_name} como componente NLP principal")
    print(f"2. Combinar con modelos BERT para mejor cobertura")
    print(f"3. Usar features TF-IDF como input para meta-modelos")
    print(f"4. Implementar ensemble voting con modelos top 3")
    print(f"5. Optimizar hiperparámetros en conjunto")
    
    print(f"\nESTADO FINAL: {status}")
    print(f"Preparado para integración en ensemble híbrido")
    
else:
    print(f"\nPROBLEMA: No se entrenaron modelos exitosamente")
    print(f"Revisar datos de entrada y configuración")
    status = "FALLIDO"

print(f"\nFinalizado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"ANALISIS NLP CON TF-IDF COMPLETADO")
print(f"Estado: {status}")
print(f"Sin data leakage - Listo para ensemble")


RESUMEN FINAL - MODELOS NLP CON TF-IDF

TRABAJO REALIZADO:
1. Carga y análisis de 134,198 textos originales
2. Corrección de data leakage eliminando 133,140 duplicados
3. Preprocesamiento avanzado de texto con NLTK
4. Vectorización TF-IDF con 1,250 features
5. Entrenamiento de 6 modelos exitosos
6. Evaluación completa con métricas detalladas

DATOS PROCESADOS:
- Dataset original: 134,198 filas
- Después de eliminar duplicados: 1,058 filas
- Después de filtrar texto válido: 1,058 filas
- Reducción total: 99.2%

VECTORIZACION TF-IDF:
- Features máximas configuradas: 8,000
- Features reales generadas: 1,250
- N-gramas utilizados: (1, 2)
- Sparsity de la matriz: 99.3%

MODELOS ENTRENADOS Y RESULTADOS:
1. Logistic Regression:
   - F1-Score: 0.8362
   - Accuracy: 0.8208
   - Tiempo: 0.0s
2. SVM Linear:
   - F1-Score: 0.8261
   - Accuracy: 0.8113
   - Tiempo: 0.4s
3. SVM RBF:
   - F1-Score: 0.8511
   - Accuracy: 0.8349
   - Tiempo: 0.5s
4. Multinomial NB:
   - F1-Score: 0.8174
   - Accuracy: